# Merger and Acquisition Simulation Model

Date: February 23rd, 2020

Author: Armand Tossou

## Purpose  

The purpose of this project is to simulate the process of merger and acquisition (i.e., M&A) between three firms and project their ex-post market valuation, as well as the resulting margin from the operation. Merger  simulation  models  predict  post-merger  prices  based  on  information  about  a  set  of  pre-merger  market  conditions  and  certain  assumptions  about  the behavior of the firms in the relevant market.

This project replicates the analysis in the seminal M&A paper by __Rubinfeld and Epstein (2001)__. Below is the full reference of the paper:

    Rubinfeld, Daniel L., and Roy J. Epstein. "Merger simulation: A simplified approach with new applications." (2001).
    Link: https://escholarship.org/uc/item/2sq9s8c8


## Methodology  

Rubinfeld and Epstein (2001) introduced PCAIDS, a highly flexible "calibrated-demand" merger simulation methodology that is based on a simplified version of AIDS. PCAIDS can be implemented using market shares and two price elasticities; scanner or transaction-level data are not required. 

The advantage of this approach is that the model can be calibrated with relatively little information. In particular, we need to know the following:  
- the market shares of the firms in the relevant market;  
- elasticity of market demand;  
- own price elasticity for one firm; and
- efficiency gains (claimed by firms).

Once, we know these things, we can simulate the market outcome before and after the merger. If we do not know one or more parameters, we can do sensitivity analysis: simply let the parameter run over a vector of values.

## Assumptions  

Every model makes some assumptions on which the validity of its outcome depends. Therefore, predictions from the model need to be interpreted within this context. This model relies on the following set of three assumptions:  
- _Proportionality_: if a firm raises its price, it loses market share; this lost market share is allocated to the other firms in the industry proportionally to these firms’ market shares;  
- _Homogeneity_: if all firms in the market raise their price by the same percentage, market shares are unaffected
- _Adding-up_: market shares of all firms (brands) in the market add up to 1.

The rest of the project will be organized as follows:  
1. First, we'll import all the necessary Python packages for our analysis;  
2. We'll formulate the _demand_ function to be used to measure each firm's market share;  
3. We'll calibrate the model; and  
4. We'll estimate the model, by replicating the example in Table 1 on page 895 of Epstein and Rubinfeld (2001).

## Importing Python libraries

In [1]:
from scipy import optimize,arange
from numpy import array
import matplotlib.pyplot as plt
from math import log
%matplotlib inline

## Demand  function

Let's suppose Firm 1, Firm 2 and Firm 3 are contemplating an M&A. One Firm (i.e., Firm 3) absorbs the other two; that Firm does not merge. 

First, we define the market share of firm $i$ as follows defined as:  

$ s_i = \frac{p_i * q_i}{P * Q} $

where:  
- $p_i$ represents the firm’s price;  
- $q_i$ denotes its quantity;  
- $Q$ is the total output on the market; and 
- $P$ denotes its aggregate industry price index. 

Next, we assume that this price index is given by the following formula: 

$ln(P) = \sum_{i=1}^3 = s_i ln(p_i)$

Therefore, we can write Firm i’s demand in this model in terms of its market share as:

(D) $$\begin{aligned}
s_i = a_i + b_{ii} ln(p_i) + b_{ij} ln(p_j) + b_{ik} ln(p_k)
\end{aligned}$$

where $a$ and $b$ are parameters to be estimated.

## Model calibration  considerations

As proven by Rubinfeld and Epstein (2001), this demand function has the following useful property. 

Assuming we know:  
1. The market demand elasticity: $\varepsilon = \frac{\partial ln(Q)}{\partial ln(P)} $;  
2. Each firm's own price elasticity $\varepsilon_{ii} = \frac{\partial ln(q_i)}{\partial ln(p_i)} $; and  
3. The market shares: $s_i \geq 0$ for Firm $i = 1, \cdots, n$, with $\sum_{i=1}^n = 1$.

Given the above, if we are willing to assume $(P)$, $(H)$ and $(A)$, then we know all coefficients $b_{ij}$ and demand elasticities $\varepsilon_{ij}$.  

One way to find Firm $i$’s own price elasticity is to estimate its price cost margin, $\mu_i$. Since firms set their price (i.e., cost margin) satisfying $\mu_i = \frac{−1}{\varepsilon_{ii}}$, we know $\varepsilon_{ii}$ if we know the price cost margin.

NOTE: Also, if we don't know a parameter very well, then we can define an interval for it and do the merger simulation for each value in the interval. This will give us a good idea of how sensitive the simulation is for the value of this parameter.

## Model estimation  

We estimate the model, by replicating the example in Table 1 on page 895 of Epstein and Rubinfeld (2001).

In [2]:
# Define a Python class. We use it to below to get decent tables with results in the notebook.

class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in
        IPython Notebook. """

    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")

            for col in row:
                html.append("<td>{0}</td>".format(col))

            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

We create Python dictionaries for each Firm’s market share and elasticity, such that we can refer to Firm $i$’s market share as `market_share['firmi']`. Also, we check that the market shares are all between 0 and 1. Further, a Firm’s own price elasticity is always larger (in absolute value) than the demand elasticity. Indeed, when a firm raises its price by 1%, a consumer can either not buy the product at all or buy it from another firm.

The price elasticity equations are derived in the paper:

(E) $$\begin{aligned}
    e_{ij} = 
    \begin{cases}
-1 + \frac{b_{ii}}{s_i} + s_i * (\varepsilon + 1), \;\;  & \text{if i=j} \\ \frac{b_{ij}}{s_i} + s_j * (\varepsilon + 1), \;\;  & \text{otherwise}. 
    \end{cases}
   \end{aligned}$$

<font color='red'> IMPORTANT: </font> In this example, we assign 20%, 30%  and 50% market shares, respectively, for Firm 1, Firm 2 and Firm 3.

In [3]:
market_share = {} # creates a dictionary with market shares
elasticity = {}   # dictionaries of firm level own and cross elasticities

firms = ['firm1','firm2','firm3'] # names of the firms in a python list
market_share['firm2'] = 0.3
market_share['firm3'] = 0.5
market_share['firm1'] = 1 - sum(market_share[firm] for firm in ['firm2','firm3'])

for firm in firms:
    if (market_share[firm] >= 0 and market_share[firm] <= 1):
        print("market share of firm: "+ firm + " is given by %1.2f" % market_share[firm])
    else: print("something wrong with the market share of the firm: "+firm)

market_elasticity = -1.0 # market demand elasticity
elasticity['firm1','firm1'] = -3.0 # own elasticity of firm1

if market_elasticity < elasticity['firm1','firm1']:
    print("error: a firm's own elasticity exceeds (in absolute value) the market elasticity")

market share of firm: firm1 is given by 0.20
market share of firm: firm2 is given by 0.30
market share of firm: firm3 is given by 0.50


Now we can calculate the relevant parameters of the demand structure (see the paper for the derivation of these expressions). We create a dictionary to store the $b_{ij}$ coefficients in demand system (D) above. With the $b_{ij}$’s, we calculate the firm level (i.e., own and cross) elasticities $\varepsilon_{ij}$. The assumption is that the $b_{ij}$ coefficients are exogenous (i.e., they don't change after the merger). As the merger affects firms’ market shares, it does change the firm elasticities. 

In [4]:
b = {} # dictionary of b-coefficients of demand system (D) above
b['firm1','firm1'] = market_share['firm1']*(elasticity['firm1','firm1']+1-market_share['firm1']*(market_elasticity+1))

for firm in firms:
    b[firm,firm]=(market_share[firm]*(1-market_share[firm]))/(market_share['firm1']*(1-market_share['firm1']))*b['firm1','firm1']

for one_firm in firms:
    for other_firm in firms:
        if one_firm == other_firm:
            b[one_firm,one_firm]=(market_share[one_firm]*(1-market_share[one_firm]))/(market_share['firm1']*(1-market_share['firm1']))*b['firm1','firm1']
        else:
            b[one_firm,other_firm] = - market_share[one_firm]/(1-market_share[other_firm])*b[other_firm,other_firm]

for one_firm in firms:
    for other_firm in firms:
        if one_firm == other_firm:
            elasticity[one_firm,one_firm] = -1 + b[one_firm,one_firm]/market_share[one_firm]+market_share[one_firm]*(market_elasticity+1)
        else:
            elasticity[one_firm,other_firm] = b[one_firm,other_firm]/market_share[one_firm]+market_share[other_firm]*(market_elasticity+1)

table = ListTable()
table.append(['b-coeff.', 'firm1','firm2','firm3'])
for one_firm in firms:
    row = []
    row.append(one_firm)
    for other_firm in firms:
        row.append("%0.3f" % b[one_firm,other_firm])
    table.append(row)
print("table of b coefficients:")

table

table of b coefficients:


b-coeff.,firm1,firm2,firm3
firm1,-0.400,0.150,0.250
firm2,0.150,-0.525,0.375
firm3,0.250,0.375,-0.625


Now, let's get the elasticities:

In [5]:
table = ListTable()
table.append(['elast.', 'firm1','firm2','firm3'])
for one_firm in firms:
    row = []
    row.append(one_firm)
    for other_firm in firms:
        row.append("%0.3f" % elasticity[one_firm,other_firm])
    table.append(row)
print("table of elasticities:")

table

table of elasticities:


elast.,firm1,firm2,firm3
firm1,-3.000,0.750,1.250
firm2,0.500,-2.750,1.250
firm3,0.500,0.750,-2.250


A profit maximizing Firm $i$ sets its margin equal to $\mu_i = \frac{−1}{e_{ii}}$. This can be derived as follows. A firm chooses its price level $p_i$ to maximize profits:

$$\begin{aligned}
\pi = p_i q_i(p_i,p_{-i}) - c_i(q_i(p_i,p_{-i}))
\end{aligned}$$

First order condition are then:  

$$\begin{aligned}
q_i + (p_i - c_q) \frac{\partial q_i}{\partial p_i} = 0
\end{aligned}$$

With further rearrangement, we get:  

$$\begin{aligned}
\mu_i = \frac{p_i c_q}{p_i} = \frac{dp_i * q_i}{dq_i * p_1} = \frac{-1}{e_{ii}}
\end{aligned}$$

Hence we can calculate the margin for each Firm as follows:

In [6]:
margin = {}
for firm in firms:
    margin[firm] = -1.0/elasticity[firm,firm]

The paper shows that the ex-post merger outcome in terms of market share, $s^p_j$, price cost margin, $\mu^p_j$, and price change $\sigma_j = \frac {p^p_j − p_j}{p_j}$ (9 variables with 3 Firms) satisfies the following 9 equations, where the efficiency gain is defined as:

$$\begin{aligned}
\gamma_i = \frac{c^p_i - c_i}{c_i}
\end{aligned}$$

$\gamma_i$ is the percentage change in Firm $i$’s costs $c_i$ due to the merger.

Further, we have the elasticity equations (E) above with post-merger market shares, denoted $s^p_j$.

Details are in the paper. The following is a sketch of the proof of the equation ebove. The equation for $s^p_i$ follows from writing the (D) equations for both $s_i$ (as above) and for $s^p_i$. Then, we take the difference of these equations and note that $1 + \delta_i=\frac{p^p_i}{p_i}$ and $ln(p^p_i)−ln(p_i)=ln(1+\delta_i)$.

For the non merging firm we have $\mu_3 = \frac{−1}{e^p_{33}}$ as before. Now, consider the first order condition for $p_j$ for the merged firm (consisting of $j$ and $k$):

$$\begin{aligned}
\frac{\partial q_j}{\partial p_j}(p_j - c_j) + q_j + \frac{\partial q_k}{\partial p_j}(p_k - c_k)
\end{aligned}$$

which can be re-written as:

$$\begin{aligned}
s^p_j = -\varepsilon^p_{ii} s^p_j \mu^p_j - \varepsilon^p_{kj} s^p_k \mu^p_k
\end{aligned}$$

Finally, the expression for $\mu^p_i$ follows from the definitions of $\gamma_i$, $\delta_i$ and $\mu_i$.

In [7]:
merging_firms = ['firm1','firm2']                      # list of the merging firms
nonmerging_firms = list(set(firms)-set(merging_firms)) # list of non-merging firms
efficiency_gains = {} # dictionary with efficiency gains --due to the merger-- for the firms
for firm in merging_firms:
    efficiency_gains[firm] = 0.0
for firm in nonmerging_firms:
    efficiency_gains[firm] = 0.0

We create Python dictionaries for post merger market shares ($s^p_i$), elasticities ($e^p_{ij}$), margins ($\mu^p_i$) and price changes ($\delta_i$). Then we create a vector function with all the equations above. We solve for the values of $s^p_i$, $e^p_{ij}$, $\mu^p_i$, and $\delta_i$ such that this vector function equals zero (all equations are satisfied).

We form a list of equations and append it with new equations. For a given guess of the values $s^p_i$, $e^p_{ij}$, $\mu^p_i$, and $\delta_i$ equations returns the extent to which the equations are satisfied. If equations=0, all equations are satisfied and we find the post merger equilibrium.

In [8]:
post_marketshare = {}
post_elasticity = {}
post_margin = {}
price_change = {}

def vector_function(post_marketshare,post_elasticity,post_margin,price_change):
    equations = []
    for firm in sorted(firms):
        equations.append(post_marketshare[firm]-(market_share[firm]+sum([b[firm,other_firm]*log(1+price_change[other_firm]) for other_firm in firms])))
        equations.append(post_margin[firm] - (1-(1+efficiency_gains[firm])/(1+price_change[firm])*(1-margin[firm])))
    for firm in sorted(nonmerging_firms):
        equations.append(post_margin[firm] + 1/post_elasticity[firm,firm])
        equations.append(post_elasticity[firm,firm] - (-1 + b[firm,firm]/post_marketshare[firm]+post_marketshare[firm]*(market_elasticity+1)))
    for firm in sorted(merging_firms):
        equations.append(post_marketshare[firm]+sum(post_elasticity[other_firm,firm]*post_marketshare[other_firm]*post_margin[other_firm] for other_firm in sorted(merging_firms)))
        for other_firm in sorted(merging_firms):
            equations.append(post_elasticity[firm,other_firm] - (-(firm==other_firm) + b[firm,other_firm]/post_marketshare[firm]+post_marketshare[other_firm]*(market_elasticity+1)))
    return equations

As the solver works with a vector $x$ for the values (not a collection of dictionaries), we define a wrapper function which turns $x$ to the values in the dictionaries. Once we have the solution $x$, we ‘un-wrap’ it into values for our dictionaries.

In [9]:
def wrapper_function(x): # the solver used below needs a vector x as input, not a collection of dictionaries
    counter = 0
    for dict in [post_marketshare,post_margin,price_change]:
        for firm in sorted(firms):
            dict[firm] = x[counter]
            counter +=1
    for firm in sorted(nonmerging_firms):
        post_elasticity[firm, firm] = x[counter]
        counter +=1
    for firm in sorted(merging_firms):
        for other_firm in sorted(merging_firms):
            post_elasticity[firm, other_firm] = x[counter]
            counter +=1
    return vector_function(post_marketshare,post_elasticity,post_margin,price_change)

def unwrap(x):   # once we have the equilibrium solution as a vector x, we go back to dictionaries
    counter = 0  # as they are easier to work with
    for dict in [post_marketshare,post_margin,price_change]:
        for firm in sorted(firms):
            dict[firm] = x[counter]
            counter +=1
    for firm in sorted(nonmerging_firms):
        post_elasticity[firm, firm] = x[counter]
        counter +=1
    for firm in sorted(merging_firms):
        for other_firm in sorted(merging_firms):
            post_elasticity[firm, other_firm] = x[counter]
            counter +=1
    return [post_marketshare,post_elasticity,post_margin,price_change]

The solver needs an initial guess for the post merger solution. We use the pre-merger values as initial guess.

In [10]:
def initial_value(): # initial value based on before merger values
    x = []
    for dict in [market_share,margin]:
        for firm in sorted(firms):
            x.append(dict[firm])
    for firm in sorted(firms):
        x.append(0.0) # price change
    for firm in sorted(nonmerging_firms):
        x.append(elasticity[firm, firm])
    for firm in sorted(merging_firms):
        for other_firm in sorted(merging_firms):
            x.append(elasticity[firm, other_firm])
    return x

We use the `fsolve` routine to find the values where `vector_function` equals 0 (i.e., where all equations hold). We solve this via `wrapper_function` as `fsolve` expects a vector $x$ as input (not a list of dictionaries). We print the relevant post merger values in a table.

In [11]:
# define a function for after merger values:
def after_merger_values():
    outcome = unwrap(optimize.fsolve(wrapper_function, initial_value()))
    table = ListTable()
    table.append(['firm', 'marketshare','margin','price increase'])
    for firm in firms:
        table.append([firm, "%0.3f" % outcome[0][firm],"%0.3f" % outcome[2][firm],"%0.3f" % outcome[3][firm]])
    print("table of post merger outcomes:")
    return table

# call after merger values function
after_merger_values()

table of post merger outcomes:


firm,marketshare,margin,price increase
firm1,0.174,0.414,0.138
firm2,0.281,0.425,0.108
firm3,0.546,0.466,0.041


__Conclusions__:  
The results from the estimated model are intuitive. As one would expect, a merger between Firms producing substitutes with no efficiency gains leads to higher prices for all Firms. The market share of the Firm that does not merge (i.e., Firm 3) increases due to the merger, as prices increase more for the merging Firms.

The end! :)